In [1]:
import pandas as pd
import numpy as np

In [2]:
matches = pd.read_csv('/Users/tliu/Desktop/Erdos Project/Data/matches.csv')

In [3]:
matches.head()

,player1,player2,score1,score2,best_of,tournament_id,date,year
0,Colin Roscoe,Jackie Rea,9,6,17,753,NaN,1982
1,Tommy Murphy,Clive Everton,9,4,17,753,NaN,1982
2,Vic Harris,Marcus Owen,9,4,17,753,NaN,1982
3,Bob Harris,Graham Cripsey,9,6,17,753,NaN,1982
4,Geoff Foulds,Matt Gibson,9,3,17,753,NaN,1982


In [4]:
tournaments = list(matches.tournament_id.unique())


In [5]:
matches[matches['tournament_id'] == tournaments[1040]]

,player1,player2,score1,score2,best_of,tournament_id,date,year
111102,Xing Zihao,Fergal O'Brien,4,3,7,5848,2023-11-02,2023
111103,Cao Yupeng,David Lilley,4,2,7,5848,2023-10-30,2023
111104,Zhou Yuelong,Anthony Hamilton,4,0,7,5848,2023-10-30,2023
111105,Kyren Wilson,Mostafa Dorgham,4,2,7,5848,2023-12-11,2023
111106,Gary Wilson,Elliot Slessor,4,3,7,5848,2023-12-11,2023
...,...,...,...,...,...,...,...,...
111222,Noppon Saengkham,Sanderson Lam,5,3,9,5848,2023-12-15,2023
111223,John Higgins,Tom Ford,5,3,9,5848,2023-12-15,2023
111224,Gary Wilson,Zhou Yuelong,6,5,11,5848,2023-12-16,2023
111225,Noppon Saengkham,John Higgins,6,3,11,5848,2023-12-16,2023


In [6]:
#Using matches history before tournament xxxx to predict the results of tournament xxxx
from elo_v3 import elo_rating
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


#Ks represent K-factors we use
#nth means nth tournament among 1085 tournaments
Ks = [4, 6, 8, 15, 20]
nth = [750, 999, 1040, 1080]
scores1 = np.zeros((5,4))


i=0
for K in Ks:
    j=0
    for n in nth:
        past_tournaments = tournaments[:n]
        current = tournaments[n]
        past_index = matches['tournament_id'].map(lambda x: x in past_tournaments)
        

        matches_past = matches[past_index]
        matches_future = matches[matches['tournament_id'] == current]

        #Update statistics of players
        elo = elo_rating()
        elo.update_stats(matches = matches_past, K_factor = K, ratio = 0.1)

        #Store the actual result in y
        matches_future['result'] = 0  
        y = np.array(matches_future['result'])

        #Compute expected win_rate
        X = matches_future[['player1', 'player2', 'best_of']]
        _, expected = elo.predict(X)

        #Compute the actual frame_rate
        actual = np.array(matches_future['score1']/(matches_future['score1']+matches_future['score2']))

        #Record the accuracy score
        scores1[i,j] = mean_squared_error(expected, actual)
        
        j += 1
    i+=1




/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_3919/720966034.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0
/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(cop

In [ ]:
#Using matches history before tornament xxxx to predict the results of tournament xxxx
from elo_v3 import elo_rating
from sklearn.metrics import mean_squared_error


#Ks represent K-factors we use
#nth means nth tournament among 1085 tournaments
Ks = [2,3,5,7,8]
nth = [750, 999, 1040, 1080]
scores2 = np.zeros((5,4))


i=0
for K in Ks:
    j=0
    for n in nth:
        past_tournaments = tournaments[:n]
        current = tournaments[n]
        past_index = matches['tournament_id'].map(lambda x: x in past_tournaments)
        

        matches_past = matches[past_index]
        matches_future = matches[matches['tournament_id'] == current]

        #Update statistics of players
        elo = elo_rating()
        elo.update_stats(matches = matches_past, K_factor = K, ratio = 0.1)

        #Store the actual result in y
        matches_future['result'] = 0  
        y = np.array(matches_future['result'])

        #Compute expected win_rate
        X = matches_future[['player1', 'player2', 'best_of']]
        _, expected = elo.predict(X)
        print(expected)

        #Compute the actual frame_rate
        actual = np.array(matches_future['score1']/(matches_future['score1']+matches_future['score2']))
        print(actual)

        #Record the accuracy score
        from sklearn.metrics import accuracy_score
        scores2[i,j] = mean_squared_error(expected, actual)
        
        j += 1
    i+=1


In [8]:
scores_df = pd.concat([pd.DataFrame(scores1), pd.DataFrame(scores2)])

In [9]:
scores_df['K-factor'] = [4, 6, 8, 15, 20, 2,3,5,7,8]

In [10]:
scores_df.set_index('K-factor', inplace=True)

In [11]:
scores_df.rename(columns = {
    0: 750,
    1: 999,
    2: 1040,
    3: 1080
})

,750,999,1040,1080
K-factor,,,,
4,0.045110,0.049717,0.071264,0.033808
6,0.048516,0.052047,0.071185,0.031674
8,0.050988,0.053987,0.071634,0.031000
15,0.057199,0.058690,0.074387,0.033496
20,0.060654,0.061335,0.076790,0.036670
2,0.040819,0.047587,0.072804,0.038124
3,0.042967,0.048502,0.071767,0.035694
5,0.046935,0.050936,0.071129,0.032485
7,0.049832,0.053068,0.071372,0.031193


In [12]:
average_score = scores_df.mean(axis=1)
average_score

K-factor
4     0.049975
6     0.050855
8     0.051902
15    0.055943
20    0.058862
2     0.049834
3     0.049733
5     0.050372
7     0.051366
8     0.051902
dtype: float64